## Modelo 2

In [ ]:
import os
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import alpaca_trade_api as tradeapi
from gym import spaces
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import DummyVecEnv


# ======================================
# 1. Discrete Trading Environment (Buy/Hold/Sell)
# ======================================
class DiscreteTradingEnvDQN(gym.Env):
    metadata = {'render.modes': ['human']}

    def _init_(
        self,
        alpaca_api_key,
        alpaca_api_secret,
        alpaca_base_url,
        ticker="F",
        initial_cash=100_000,
        lookback_window=1,
        start_date="2022-01-01",
        end_date="2022-06-30",
        timeframe='1Hour'
    ):
        super(DiscreteTradingEnvDQN, self)._init_()

        self.alpaca_api_key = alpaca_api_key
        self.alpaca_api_secret = alpaca_api_secret
        self.alpaca_base_url = alpaca_base_url
        self.ticker = ticker
        self.initial_cash = initial_cash
        self.lookback_window = lookback_window
        self.start_date = start_date
        self.end_date = end_date
        self.timeframe = timeframe

        self.api = tradeapi.REST(
            self.alpaca_api_key,
            self.alpaca_api_secret,
            self.alpaca_base_url,
            api_version='v2'
        )

        # Actions: 0 = Sell All, 1 = Hold, 2 = Buy (all-in with available cash)
        self.action_space = spaces.Discrete(3)

        # Observation includes OHLCV for lookback_window steps, plus shares held and cash
        obs_size = (self.lookback_window * 5) + 2
        self.observation_space = spaces.Box(low=0.0, high=np.inf, shape=(obs_size,), dtype=np.float32)

        self.all_data = self._fetch_historical_data()
        self.reset()

    def _fetch_historical_data(self):
        bars = self.api.get_bars(
            symbol=self.ticker,
            timeframe=self.timeframe,
            start=self.start_date,
            end=self.end_date
        )
        data = pd.DataFrame([{
            'time': bar.t,
            'open': bar.o,
            'high': bar.h,
            'low': bar.l,
            'close': bar.c,
            'volume': bar.v
        } for bar in bars])
        data.set_index('time', inplace=True)
        data.sort_index(inplace=True)
        data.reset_index(drop=True, inplace=True)
        return data

    def reset(self):
        self.data = self.all_data.copy()
        self.current_step = self.lookback_window
        self.cash = float(self.initial_cash)
        self.shares = 0
        self.done = False
        self.episode_reward = 0.0
        self.consecutive_hold_steps = 0
        return self._next_observation()

    def _next_observation(self):
        start = self.current_step - self.lookback_window
        end = self.current_step
        window_data = self.data.iloc[start:end]

        obs_data = []
        for _, row in window_data.iterrows():
            obs_data.extend([
                row['open'],
                row['high'],
                row['low'],
                row['close'],
                row['volume'],
            ])

        # Add shares held and cash as part of the observation
        obs_data.append(self.shares)
        obs_data.append(self.cash)

        return np.array(obs_data, dtype=np.float32)

    def step(self, action):
        old_portfolio_value = self._get_portfolio_value()

        # Current price
        row = self.data.iloc[self.current_step]
        current_price = row['close']

        # Execute action
        if action == 0:
            # Sell all shares
            self.cash += self.shares * current_price
            self.shares = 0
            self.consecutive_hold_steps = 0  # Reset hold counter
        elif action == 1:
            # Hold (do nothing)
            self.consecutive_hold_steps += 1
        elif action == 2:
            # Buy as many shares as possible with the available cash
            if current_price > 0:
                shares_to_buy = int(self.cash // current_price)
                self.cash -= shares_to_buy * current_price
                self.shares += shares_to_buy
            self.consecutive_hold_steps = 0  # Reset hold counter

        hold_penalty = 0
        if self.consecutive_hold_steps > 50:  # Define the threshold
            hold_penalty = -5  # Penalty for holding too long

        # Move forward in time
        self.current_step += 1
        if self.current_step >= len(self.data):
            self.done = True

        # Compute new portfolio value
        new_portfolio_value = self._get_portfolio_value()
        reward = new_portfolio_value - old_portfolio_value + hold_penalty


        # If portfolio reaches at least 120% of initial cash, end episode immediately and give big reward
        if new_portfolio_value >= (self.initial_cash * 1.2):
            # Add 20000 to the reward
            reward += 20000
            self.done = True

        
        self.episode_reward += reward

        print(f"Step: {self.current_step}, Action: {action}, Shares: {self.shares}, "
              f"Cash: {self.cash:.2f}, Portfolio: {new_portfolio_value:.2f}, Reward: {reward:.2f}")

        if self.done:
            print(f"Episode finished. Total Episode Reward: {self.episode_reward:.2f}")

        obs = self._next_observation() if not self.done else np.zeros(self.observation_space.shape, dtype=np.float32)
        info = {'portfolio_value': new_portfolio_value}
        return obs, reward, self.done, info

    def _get_portfolio_value(self):
        current_price = self.data.iloc[self.current_step - 1]['close']
        return self.cash + (self.shares * current_price)

    def render(self, mode='human'):
        portfolio_value = self._get_portfolio_value()
        print(f"Step: {self.current_step}, Shares: {self.shares}, Cash: {self.cash:.2f}, Portfolio: {portfolio_value:.2f}")